#### Load data

In [5]:
with open('log/IIIHTTP.log','r') as InputData:
    APPdata = InputData.readlines()
    APPdata = [i.strip().split('\t') for i in APPdata]
    #For BackdroundAPP.txt format below 
    #APPdata = [i.strip().split(',') for i in APPdata]
    APPdata = list(filter(lambda x:'#' not in x[0],APPdata))

In [6]:
with open('log/IIIHTTP3.log','r') as InputData2:
    Testdata = InputData2.readlines()
    Testdata = [i.strip().split('\t') for i in Testdata]
    Testdata = list(filter(lambda x:'#' not in x[0],Testdata))

In [7]:
def similarity(test_data,whiteList_data):
    score = 0
    for key in fingerprint_keys_index:
        for key_index in fingerprint_keys_index[key]:
            if test_data[key_index] in whiteList_data[key]:
                score += 1
    #if score > 0:
    #    WhiteListUpdate(test_data,whiteList_data)
    return score

In [8]:
def WhiteListUpdate(data):
    pass

In [9]:
def outgoingInfo(data):
    pass

#### Training the white list fingerprint

In [10]:
fingerprint_keys_index={'ip':[4],'host':[8],'userAgent':[12]}
trainedList = {}
for index in range(len(APPdata)):
    if APPdata[index][12].strip() != "'-'":
        label = APPdata[index][12].replace('\'','').strip()
        if label not in trainedList:
            tmp_dict = {'ip':[APPdata[index][4].replace('\'','').strip()],
                        'host':[APPdata[index][8].replace('\'','').strip()],
                        'userAgent':[label]}
            trainedList.setdefault(label)
            trainedList[label] = tmp_dict
        else:
            for key in fingerprint_keys_index:
                for key_index in fingerprint_keys_index[key]:
                    tmp_APPdata = APPdata[index][key_index].replace('\'','').strip()
                    if  tmp_APPdata not in trainedList[label][key]:
                        trainedList[label].setdefault(key,[]).append(tmp_APPdata)

In [11]:
toBeDetermined = []
NoHost =[]
for test_index in range(len(Testdata)):
    similarityScore = 0
    if Testdata[test_index][8] == '-':
        NoHost.append(Testdata[test_index])
    else:
        if Testdata[test_index][12] in trainedList:
            whiteList = trainedList[Testdata[test_index][12]]
            similarityScore = similarity(Testdata[test_index],whiteList)
        else:
            max_score = 0
            likelihoodAPP = []
            for label in trainedList:
                similarityScore = similarity(Testdata[test_index],trainedList[label])
                if similarityScore > max_score:
                    likelihoodAPP = Testdata[test_index]
                    max_score = similarityScore
                elif (similarityScore > 0) and (similarityScore < max_score):
                    likelihoodAPP.append(Testdata[test_index])
            if max_score == 0:
                toBeDetermined.append(Testdata[test_index])
            #print(similarityScore,Testdata[test_index][12],likelihoodAPP)

In [13]:
print(toBeDetermined[:10])

[['1521514507.431700', 'CQpVSQ37fy6QUJNBK1', '140.92.39.43', '4352', '37.252.227.51', '80', '1', 'GET', 'ping3.teamviewer.com', '/din.aspx?s=00000000&client=DynGate&rnd=400557012&p=10000001', '-', '-', 'Mozilla/4.0 (compatible; MSIE 6.0; DynGate)', '0', '0', '-', '-', '-', '-', '(empty)', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['1521514507.456668', 'CJWVZY2CTRtr2fgCDc', '140.92.211.22', '50873', '162.220.223.28', '80', '1', 'GET', 'ping3.teamviewer.com', '/din.aspx?s=00000000&id=492665042&client=DynGate&p=10000001', '-', '-', 'Mozilla/4.0 (compatible; MSIE 6.0; DynGate)', '0', '0', '-', '-', '-', '-', '(empty)', '-', '-', '-', '-', '-', '-', '-', '-', '-'], ['1521514507.592128', 'CMoC274tLuKCNJiPSk', '140.92.71.152', '55823', '185.188.32.4', '80', '1', 'GET', 'master10.teamviewer.com', '/din.aspx?s=00000000&id=516118294&client=DynGate&p=10000001', '-', '-', 'Mozilla/4.0 (compatible; MSIE 6.0; DynGate)', '0', '0', '-', '-', '-', '-', '(empty)', '-', '-', '-', '-', '-', '-', '-', 

In [14]:
print(len(toBeDetermined))

17063
